Download the dataset

In [4]:
# !pip install wget

import wget


# Replace with the actual file URL you want to download
file_url = "https://zenodo.org/records/10375902/files/sistan_part1_hps.zip?download=1"
output_file = wget.download(file_url)
print(f"Downloaded file: {output_file}")

# ## with import requests
# response = requests.get(file_url, stream=True)
# with open("downloaded_file.zip", "wb") as file:
#     for chunk in response.iter_content(chunk_size=1024):
#         file.write(chunk)
# print("Downloaded file: downloaded_file.zip")


Downloaded file: sistan_part1_hps.zip


Unzip

In [11]:
import zipfile
import os

# Path to the downloaded ZIP file
zip_file_path = output_file # 'downloaded_file.zip'  # Replace with the actual path to your ZIP file

# Directory to extract the GeoJSON file
output_directory = 'extracted_files'
os.makedirs(output_directory, exist_ok=True)

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zf:
    # List the contents of the ZIP file
    zip_contents = zf.namelist()
    
    # Find the GeoJSON file
    geojson_file = next((f for f in zip_contents if f.lower().endswith('.geojson')), None)
    
    if geojson_file:
        # Extract only the GeoJSON file to the output directory
        zf.extract(geojson_file, output_directory)
        geojson_data = os.path.join(output_directory, geojson_file)
        print(f"Extracted GeoJSON file to: {geojson_data}")
    else:
        print("No GeoJSON file found in the ZIP archive.")


Extracted GeoJSON file to: extracted_files\sistan_part1_hps.geojson


Display on leaflet

In [12]:
geojson_data

'extracted_files\\sistan_part1_hps.geojson'

In [13]:
import geopandas as gpd

# Load the GeoJSON file
gdf = gpd.read_file(geojson_data)

# Extract the first feature as a GeoDataFrame
first_feature_gdf = gdf.iloc[[0]]  # iloc[0] gets the first feature; using [[0]] preserves it as a GeoDataFrame


In [17]:
first_feature_gdf

,AH Date From,AH Date To,Address,Address Type,Administrative Division,Administrative Division Type,Archaeological From Date,Archaeological To Date,Assessment Activity Date,Assessment Activity Type,...,Spatial Coordinates Ref System,Surficial Geology Type,Threat Category,Threat Cause Type,Threat Inference Making Assessor Name - Actor,Threat Probability,Tidal Range,Topography Type,Wave Climate,geometry
0,None,None,None,None,None,None,None,None,2023-09-18,Archaeological Assessment (Image Interpretation),...,None,None,Natural,Water and/or Wind Action,Bijan Rouhani,Probable,None,None,None,POINT (62.09252 31.03259)


Display

In [14]:
import folium

# Get the centroid of the first feature for centering the map
centroid = first_feature_gdf.geometry.centroid.iloc[0]
map_center = [centroid.y, centroid.x]

# Create a folium map centered at the centroid of the first feature
my_map = folium.Map(location=map_center, zoom_start=12)

# Add the first feature to the map using GeoJSON
folium.GeoJson(first_feature_gdf.__geo_interface__).add_to(my_map)

# Display the map
my_map


C:\Users\Thomas Huet\AppData\Local\Temp\ipykernel_18380\1187032492.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = first_feature_gdf.geometry.centroid.iloc[0]


Select the HP

In [24]:
import ipywidgets as widgets
from IPython.display import display
import folium

# Create a list of unique EAMENA IDs
eamena_ids = gdf['EAMENA ID'].unique()

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=eamena_ids,
    description='EAMENA ID:',
    value=eamena_ids[0],  # Default selection is the first option
    style={'description_width': 'initial'}
)

# Function to update the map based on the selected ID
def update_map(eamena_id):
    # Filter GeoDataFrame for the selected EAMENA ID
    selected_feature_gdf = gdf[gdf['EAMENA ID'] == eamena_id]

    # Get the centroid of the selected feature
    centroid = selected_feature_gdf.geometry.centroid.iloc[0]
    map_center = [centroid.y, centroid.x]

    # Create a folium map centered at the centroid of the selected feature
    my_map = folium.Map(location=map_center, zoom_start=12)

    # Add the selected feature to the map using GeoJSON
    folium.GeoJson(selected_feature_gdf.__geo_interface__).add_to(my_map)

    # Display the map
    display(my_map)

# Observe changes in the dropdown selection and update the map
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_map(change['new'])

dropdown.observe(on_dropdown_change)

# Display the dropdown and an initial map
display(dropdown)
update_map(eamena_ids[100])  # Initial map with the first ID


Dropdown(description='EAMENA ID:', options=('EAMENA-0238205', 'EAMENA-0238206', 'EAMENA-0238207', 'EAMENA-0238…

C:\Users\Thomas Huet\AppData\Local\Temp\ipykernel_18380\3073036243.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = selected_feature_gdf.geometry.centroid.iloc[0]


C:\Users\Thomas Huet\AppData\Local\Temp\ipykernel_18380\3073036243.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = selected_feature_gdf.geometry.centroid.iloc[0]


C:\Users\Thomas Huet\AppData\Local\Temp\ipykernel_18380\3073036243.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = selected_feature_gdf.geometry.centroid.iloc[0]


C:\Users\Thomas Huet\AppData\Local\Temp\ipykernel_18380\3073036243.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = selected_feature_gdf.geometry.centroid.iloc[0]


C:\Users\Thomas Huet\AppData\Local\Temp\ipykernel_18380\3073036243.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = selected_feature_gdf.geometry.centroid.iloc[0]
